In [1]:
# --------------------------------------------------------------
#  POST-SWEEP ANALYSIS – load existing .pth checkpoints
# --------------------------------------------------------------
import torch, torch.nn as nn, torch.optim as optim
import torchvision, torchvision.transforms as transforms
import timm, numpy as np, pandas as pd, json, os, glob
import matplotlib.pyplot as plt
from tqdm import tqdm
from datetime import datetime

# ---------- 0. Paths ----------
OUTPUT_DIR = "/home/akshy_grp12"
os.makedirs(OUTPUT_DIR, exist_ok=True)


In [2]:

# ---------- 1. Data (same as training) ----------
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
])
test_set   = torchvision.datasets.CIFAR10(root='/home/akshy_grp12/dataset',
                                          train=False, download=True,
                                          transform=transform_test)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=128,
                                          shuffle=False, num_workers=2,
                                          pin_memory=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device}")


Using cuda


In [3]:

# ---------- 2. Helper: create model (same architecture) ----------
def create_vit_model(depth, mlp_ratio, embed_dim=192, num_heads=3, drop_path_rate=0.1):
    return timm.create_model(
        'vit_tiny_patch16_224', pretrained=False, num_classes=10,
        img_size=32, patch_size=4,
        embed_dim=embed_dim, depth=depth, num_heads=num_heads,
        mlp_ratio=mlp_ratio, qkv_bias=True,
        drop_path_rate=drop_path_rate
    )

# ---------- 3. Evaluation (clean test) ----------
def evaluate_clean(model):
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            pred = model(x).max(1)[1]
            correct += pred.eq(y).sum().item()
            total   += y.size(0)
    return 100. * correct / total

# ---------- 4. Find *all* checkpoints ----------
pattern = f"{OUTPUT_DIR}/outputvit_d*_mlp*_noise*.pth"
ckpt_paths = sorted(glob.glob(pattern))
print(f"Found {len(ckpt_paths)} checkpoints")


Found 904 checkpoints


In [4]:

# ---------- 5. Parse filename → hyper-params ----------
import re
def parse_ckpt(path):
    # vit_d16_mlp4.5_noise0.0.pth  → depth=16, mlp=4.5, noise=0.0
    m = re.search(r'outputvit_d(\d+)_mlp([\d\.]+)_noise([\d\.]+)\.pth', os.path.basename(path))
    if not m: raise ValueError(path)
    depth = int(m.group(1))
    mlp   = float(m.group(2))
    noise = float(m.group(3))
    return depth, mlp, noise

# ---------- 6. Evaluate every checkpoint ----------
rows = []
for p in tqdm(ckpt_paths, desc="Evaluating checkpoints"):
    depth, mlp, noise = parse_ckpt(p)
    model = create_vit_model(depth=depth, mlp_ratio=mlp)
    model.load_state_dict(torch.load(p, map_location=device))
    model.to(device)

    acc = evaluate_clean(model)                 # clean test accuracy
    params = sum(par.numel() for par in model.parameters())

    rows.append({
        "train_noise": noise,
        "depth": depth,
        "mlp_ratio": mlp,
        "embed_dim": 192,
        "num_heads": 3,
        "params": params,
        "epochs": 30,
        "test_noise": 0.0,
        "accuracy": acc
    })
    del model; torch.cuda.empty_cache()

# ---------- 7. Save master CSV ----------
master_csv = f"{OUTPUT_DIR}/vit_grid_results.csv"
pd.DataFrame(rows).to_csv(master_csv, index=False)
print(f"\nMaster CSV saved → {master_csv}")

# ---------- 8. Quick line-plot (depth vs noise) ----------
def plot_line(df):
    plt.figure(figsize=(10,6))
    for d in sorted(df['depth'].unique()):
        sub = df[df['depth']==d].sort_values('train_noise')
        plt.plot(sub['train_noise'], sub['accuracy'], 'o-', label=f'Depth {d}')
    plt.xlabel('Train Noise'); plt.ylabel('Clean Test Acc (%)')
    plt.title('ViT Accuracy vs Training Noise')
    plt.legend(); plt.grid(alpha=.3)
    out = f"{OUTPUT_DIR}/vit_line.png"
    plt.savefig(out, dpi=300, bbox_inches='tight'); plt.close()
    print(f"Line plot → {out}")

plot_line(pd.DataFrame(rows))

# ---------- 9. Surface plots (one per noise level) ----------
def surface_plots(df):
    df = df[df['test_noise']==0.0]
    for nv in sorted(df['train_noise'].unique()):
        sub = df[df['train_noise']==nv]
        depths = sorted(sub['depth'].unique())
        mlps   = sorted(sub['mlp_ratio'].unique())

        # pivot → Z matrix
        Z = sub.pivot(index='mlp_ratio', columns='depth', values='accuracy')
        Z = Z.reindex(mlps).reindex(depths, axis=1).values

        fig = plt.figure(figsize=(7,5))
        ax = fig.add_subplot(111, projection='3d')
        X, Y = np.meshgrid(depths, mlps)
        ax.plot_surface(X, Y, Z, cmap='viridis', edgecolor='none', alpha=0.9)
        ax.set_xlabel('Depth'); ax.set_ylabel('MLP Ratio'); ax.set_zlabel('Acc (%)')
        ax.set_title(f'Accuracy Surface – train_noise={nv}')
        out = f"{OUTPUT_DIR}/surface_noise_{nv}.png"
        plt.savefig(out, dpi=300, bbox_inches='tight'); plt.close()
        print(f"Surface → {out}")

surface_plots(pd.DataFrame(rows))

# ---------- 10. Save minimal metadata ----------
metadata = {
    "date": datetime.now().isoformat(),
    "grid": {
        "noise": sorted(pd.DataFrame(rows)['train_noise'].unique().tolist()),
        "depth": sorted(pd.DataFrame(rows)['depth'].unique().tolist()),
        "mlp":   sorted(pd.DataFrame(rows)['mlp_ratio'].unique().tolist())
    },
    "total_models": len(rows)
}
with open(f"{OUTPUT_DIR}/experiment_metadata.json", "w") as f:
    json.dump(metadata, f, indent=2)
print("Metadata JSON saved")

Evaluating checkpoints: 100%|██████████| 904/904 [35:12<00:00,  2.34s/it]



Master CSV saved → /home/akshy_grp12/vit_grid_results.csv
Line plot → /home/akshy_grp12/vit_line.png
Surface → /home/akshy_grp12/surface_noise_0.0.png
Surface → /home/akshy_grp12/surface_noise_0.05.png
Surface → /home/akshy_grp12/surface_noise_0.1.png
Surface → /home/akshy_grp12/surface_noise_0.15.png
Surface → /home/akshy_grp12/surface_noise_0.2.png
Surface → /home/akshy_grp12/surface_noise_0.25.png
Surface → /home/akshy_grp12/surface_noise_0.3.png
Surface → /home/akshy_grp12/surface_noise_0.35.png
Surface → /home/akshy_grp12/surface_noise_0.4.png
Surface → /home/akshy_grp12/surface_noise_0.45.png
Metadata JSON saved


More vivid graphs

In [7]:
# ==============================================================
#  POST-SWEEP: Visualization + Full Metrics + Interactive 3D
# ==============================================================
import torch, torch.nn as nn
import torchvision, torchvision.transforms as transforms
import timm, numpy as np, pandas as pd, json, os, glob, re
import matplotlib.pyplot as plt
from tqdm import tqdm
from datetime import datetime
from sklearn.metrics import roc_auc_score, precision_recall_fscore_support, accuracy_score, confusion_matrix
import plotly.graph_objects as go
from scipy.stats import sem, t

# ------------------- 0. Paths & Device -------------------
# OUTPUT_DIR = "/home/akshy_grp12/output"
# os.makedirs(OUTPUT_DIR, exist_ok=True)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using {device}")

# ------------------- 1. Data Loaders -------------------
# transform_clean = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
# ])
# test_set = torchvision.datasets.CIFAR10(root='/home/akshy_grp12/dataset', train=False,
#                                         download=True, transform=transform_clean)
# test_loader = torch.utils.data.DataLoader(test_set, batch_size=128,
#                                           shuffle=False, num_workers=2, pin_memory=True)
###################################


In [6]:
import sys
print(sys.executable)


/home/akshy_grp12/.venv/bin/python


In [ ]:


# For visualization: get 2 samples per class
class_to_idx = {name: i for i, name in enumerate(test_set.classes)}
samples_per_class = {i: [] for i in range(10)}
for img, label in test_set:
    if len(samples_per_class[label]) < 2:
        samples_per_class[label].append(img)
    if all(len(v) == 2 for v in samples_per_class.values()):
        break
vis_images = [img for cls in samples_per_class.values() for img in cls]  # 20 images
vis_labels = [i for i in range(10) for _ in range(2)]

# ------------------- 2. Noise Function -------------------
CIFAR_MEAN = (0.4914, 0.4822, 0.4465)
CIFAR_STD  = (0.2470, 0.2435, 0.2616)
_inv = transforms.Normalize(mean=[-m/s for m,s in zip(CIFAR_MEAN, CIFAR_STD)],
                            std=[1/s for s in CIFAR_STD])
_norm = transforms.Normalize(CIFAR_MEAN, CIFAR_STD)

def add_noise_in_rgb_space(x, sigma):
    if x.dim() == 3: x = x.unsqueeze(0)
    x_rgb = torch.clamp(_inv(x), 0., 1.)
    noise = sigma * torch.randn_like(x_rgb)
    x_noisy = torch.clamp(x_rgb + noise, 0., 1.)
    return _norm(x_noisy).nan_to_num_()

def denormalize(tensor):
    tensor = tensor.clone()
    for t, m, s in zip(tensor, CIFAR_MEAN, CIFAR_STD):
        t.mul_(s).add_(m)
    return torch.clamp(tensor, 0, 1)

# ------------------- 3. Model Creation -------------------
def create_vit_model(depth, mlp_ratio):
    return timm.create_model('vit_tiny_patch16_224', pretrained=False, num_classes=10,
                             img_size=32, patch_size=4, embed_dim=192, depth=depth,
                             num_heads=3, mlp_ratio=mlp_ratio, qkv_bias=True,
                             drop_path_rate=0.1)

# ------------------- 4. Load All Checkpoints -------------------
pattern = f"{OUTPUT_DIR}/outputvit_d*_mlp*_noise*.pth"
ckpt_paths = sorted(glob.glob(pattern))
print(f"Found {len(ckpt_paths)} checkpoints")

def parse_ckpt(p):
    m = re.search(r'outputvit_d(\d+)_mlp([\d\.]+)_noise([\d\.]+)\.pth', os.path.basename(p))
    return int(m.group(1)), float(m.group(2)), float(m.group(3))

# ------------------- 5. Full Metrics Function -------------------
def evaluate_full_metrics(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    all_probs = []

    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            logits = model(x)
            probs = torch.softmax(logits, dim=1)
            pred = logits.max(1)[1]

            all_preds.extend(pred.cpu().numpy())
            all_labels.extend(y.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    # Convert to numpy
    y_true = np.array(all_labels)
    y_pred = np.array(all_preds)
    y_prob = np.array(all_probs)

    # Accuracy
    acc = accuracy_score(y_true, y_pred)

    # AUC (macro + 95% CI via bootstrap)
    def bootstrap_auc(n=1000):
        aucs = []
        for _ in range(n):
            idx = np.random.choice(len(y_true), len(y_true), replace=True)
            try:
                aucs.append(roc_auc_score(y_true[idx], y_prob[idx], average='macro', multi_class='ovr'))
            except:
                pass
        return np.array(aucs)

    auc_boot = bootstrap_auc()
    auc_mean = auc_boot.mean()
    auc_ci = (auc_mean - 1.96 * sem(auc_boot), auc_mean + 1.96 * sem(auc_boot))

    # Precision, Recall, F1 (macro)
    p, r, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')

    return {
        'accuracy': acc * 100,
        'auc_mean': auc_mean * 100,
        'auc_ci_low': auc_ci[0] * 100,
        'auc_ci_high': auc_ci[1] * 100,
        'precision': p * 100,
        'recall': r * 100,
        'f1': f1 * 100
    }

# ------------------- 6. Evaluate All Models -------------------
rows = []
print("Evaluating all models with full metrics...")
for p in tqdm(ckpt_paths, desc="Models"):
    depth, mlp, noise = parse_ckpt(p)
    model = create_vit_model(depth, mlp)
    model.load_state_dict(torch.load(p, map_location=device))
    model.to(device)

    metrics = evaluate_full_metrics(model, test_loader)
    params = sum(par.numel() for par in model.parameters())

    rows.append({
        "train_noise": noise,
        "depth": depth,
        "mlp_ratio": mlp,
        "params": params,
        **metrics
    })

    del model; torch.cuda.empty_cache()

df = pd.DataFrame(rows)
csv_path = f"{OUTPUT_DIR}/vit_grid_full_metrics.csv"
df.to_csv(csv_path, index=False)
print(f"\nFull metrics CSV saved → {csv_path}")

# ------------------- 7. Feature 1: Noisy Image Visualization -------------------
def plot_noisy_samples(noise_levels=[round(x, 2) for x in np.linspace(0.0, 1.0, 21)] ):
    n_noise = len(noise_levels)
    fig, axes = plt.subplots(10, 2 + n_noise, figsize=(3*(2+n_noise), 20))
    fig.suptitle("CIFAR-10: Original vs Noisy Images", fontsize=16, fontweight='bold')

    for cls in range(10):
        idx = cls * 2
        img = vis_images[idx]  # one sample per class
        axes[cls, 0].imshow(denormalize(img).permute(1,2,0).numpy())
        axes[cls, 0].set_title(f"Class {cls} - Clean")
        axes[cls, 0].axis('off')

        axes[cls, 1].imshow(denormalize(img).permute(1,2,0).numpy())
        axes[cls, 1].set_title("Clean (dup)")
        axes[cls, 1].axis('off')

        for j, sigma in enumerate(noise_levels):
            noisy = add_noise_in_rgb_space(img.unsqueeze(0), sigma).squeeze(0)
            axes[cls, 2+j].imshow(denormalize(noisy).permute(1,2,0).numpy())
            axes[cls, 2+j].set_title(f"σ={sigma}")
            axes[cls, 2+j].axis('off')

    plt.tight_layout()
    out = f"{OUTPUT_DIR}/noisy_image_examples.png"
    plt.savefig(out, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Noisy image grid saved → {out}")

plot_noisy_samples()

# ------------------- 8. Feature 3: Interactive 3D Surface (Plotly) -------------------
def interactive_3d_surfaces(df):
    df_clean = df[df['test_noise'].isna() | (df['test_noise'] == 0.0)]
    for noise in sorted(df_clean['train_noise'].unique()):
        sub = df_clean[df_clean['train_noise'] == noise]
        depths = sorted(sub['depth'].unique())
        mlps = sorted(sub['mlp_ratio'].unique())

        Z = sub.pivot(index='mlp_ratio', columns='depth', values='accuracy')
        Z = Z.reindex(mlps).reindex(depths, axis=1).values

        fig = go.Figure(data=[go.Surface(
            x=depths, y=mlps, z=Z,
            colorscale='Viridis', showscale=True
        )])

        fig.update_layout(
            title=f'Accuracy Surface – train_noise={noise}',
            scene=dict(
                xaxis_title='Depth',
                yaxis_title='MLP Ratio',
                zaxis_title='Accuracy (%)',
                camera=dict(eye=dict(x=1.5, y=1.5, z=1.5))
            ),
            width=800, height=600
        )

        out_html = f"{OUTPUT_DIR}/3d_surface_noise_{noise}.html"
        fig.write_html(out_html)
        print(f"Interactive 3D plot → {out_html}")

interactive_3d_surfaces(df)

# ------------------- 9. Summary Table (Best per Noise) -------------------
print("\n" + "="*80)
print("BEST MODEL PER NOISE LEVEL (Accuracy)")
print("="*80)
summary = []
for noise in sorted(df['train_noise'].unique()):
    sub = df[df['train_noise'] == noise]
    best = sub.loc[sub['accuracy'].idxmax()]
    summary.append({
        'train_noise': noise,
        'depth': int(best['depth']),
        'mlp_ratio': best['mlp_ratio'],
        'accuracy': best['accuracy'],
        'auc': f"{best['auc_mean']:.1f} [{best['auc_ci_low']:.1f}-{best['auc_ci_high']:.1f}]",
        'f1': best['f1']
    })
print(pd.DataFrame(summary).round(2).to_string(index=False))

# ------------------- 10. Save Final Report -------------------
report = {
    "analysis_date": datetime.now().isoformat(),
    "total_models": len(df),
    "metrics_included": ["accuracy", "auc+ci", "precision", "recall", "f1"],
    "visualizations": {
        "noisy_images": "noisy_image_examples.png",
        "interactive_3d": [f"3d_surface_noise_{n}.html" for n in sorted(df['train_noise'].unique())]
    }
}
with open(f"{OUTPUT_DIR}/final_report.json", "w") as f:
    json.dump(report, f, indent=2)
print(f"\nFinal report → {OUTPUT_DIR}/final_report.json")

Found 904 checkpoints
Evaluating all models with full metrics...


Models:  48%|████▊     | 433/904 [2:45:09<3:04:38, 23.52s/it]

After final result csv

In [2]:
import pandas as pd, numpy as np, os
OUTPUT_DIR = "/home/akshy_grp12"   # keep your same output dir
csv_path = os.path.join(OUTPUT_DIR, "vit_grid_full_metrics.csv")
df = pd.read_csv(csv_path)
print("Loaded", csv_path, "with shape", df.shape)
display(df.head())


Loaded /home/akshy_grp12/vit_grid_full_metrics.csv with shape (904, 11)


,train_noise,depth,mlp_ratio,params,accuracy,auc_mean,auc_ci_low,auc_ci_high,precision,recall,f1
0,0.00,10,0.5,1885834,76.43,97.220481,97.215480,97.225481,76.344816,76.43,76.351263
1,0.05,10,0.5,1885834,71.46,96.307453,96.301493,96.313414,72.097264,71.46,71.492391
2,0.10,10,0.5,1885834,64.65,94.677601,94.670317,94.684884,67.447595,64.65,64.910215
3,0.15,10,0.5,1885834,62.78,94.146822,94.139002,94.154642,66.971154,62.78,63.075495
4,0.20,10,0.5,1885834,57.71,92.995612,92.987221,93.004003,62.677448,57.71,58.166470


In [5]:
import matplotlib.pyplot as plt
def plot_line(df):
    plt.figure(figsize=(10,6))
    for d in sorted(df['depth'].unique()):
        sub = df[df['depth']==d].sort_values('train_noise')
        plt.plot(sub['train_noise'], sub['accuracy'], marker='o', label=f'Depth {d}')
    plt.xlabel('Train Noise'); plt.ylabel('Clean Test Acc (%)')
    plt.title('ViT Accuracy vs Training Noise')
    plt.legend(); plt.grid(alpha=.3)
    out = os.path.join(OUTPUT_DIR, "vit_line_2.png")
    plt.savefig(out, dpi=300, bbox_inches='tight')
    plt.close()
    print("Saved:", out)

plot_line(df)


Saved: /home/akshy_grp12/vit_line_2.png


In [6]:
from mpl_toolkits.mplot3d import Axes3D
def static_surface_plots(df):
    df_clean = df.copy()
    if 'test_noise' in df.columns:
        df_clean = df_clean[df_clean['test_noise'].isna() | (df_clean['test_noise']==0.0)]
    for nv in sorted(df_clean['train_noise'].unique()):
        sub = df_clean[df_clean['train_noise']==nv]
        depths = sorted(sub['depth'].unique())
        mlps   = sorted(sub['mlp_ratio'].unique())
        Z = sub.pivot(index='mlp_ratio', columns='depth', values='accuracy')
        Z = Z.reindex(index=mlps, columns=depths).values

        fig = plt.figure(figsize=(8,6))
        ax = fig.add_subplot(111, projection='3d')
        X, Y = np.meshgrid(depths, mlps)
        ax.plot_surface(X, Y, Z, cmap='viridis', edgecolor='none', alpha=0.9)
        ax.set_xlabel('Depth'); ax.set_ylabel('MLP Ratio'); ax.set_zlabel('Acc (%)')
        ax.set_title(f'Accuracy Surface – train_noise={nv}')
        out = os.path.join(OUTPUT_DIR, f"surface_noise_{nv}.png")
        plt.savefig(out, dpi=300, bbox_inches='tight')
        plt.close()
        print("Saved:", out)

static_surface_plots(df)


Saved: /home/akshy_grp12/surface_noise_0.0.png
Saved: /home/akshy_grp12/surface_noise_0.05.png
Saved: /home/akshy_grp12/surface_noise_0.1.png
Saved: /home/akshy_grp12/surface_noise_0.15.png
Saved: /home/akshy_grp12/surface_noise_0.2.png
Saved: /home/akshy_grp12/surface_noise_0.25.png
Saved: /home/akshy_grp12/surface_noise_0.3.png
Saved: /home/akshy_grp12/surface_noise_0.35.png
Saved: /home/akshy_grp12/surface_noise_0.4.png
Saved: /home/akshy_grp12/surface_noise_0.45.png


In [7]:
import plotly.graph_objects as go
def interactive_3d_surfaces(df):
    df_clean = df.copy()
    if 'test_noise' in df.columns:
        df_clean = df_clean[df_clean['test_noise'].isna() | (df_clean['test_noise']==0.0)]
    for noise in sorted(df_clean['train_noise'].unique()):
        sub = df_clean[df_clean['train_noise'] == noise]
        depths = sorted(sub['depth'].unique())
        mlps   = sorted(sub['mlp_ratio'].unique())

        Z = sub.pivot(index='mlp_ratio', columns='depth', values='accuracy')
        Z = Z.reindex(index=mlps, columns=depths).values

        fig = go.Figure(data=[go.Surface(
            x=depths, y=mlps, z=Z,
            colorscale='Viridis', showscale=True
        )])

        fig.update_layout(
            title=f'Accuracy Surface – train_noise={noise}',
            scene=dict(
                xaxis_title='Depth',
                yaxis_title='MLP Ratio',
                zaxis_title='Accuracy (%)',
                camera=dict(eye=dict(x=1.5, y=1.5, z=1.5))
            ),
            width=900, height=700
        )

        out_html = os.path.join(OUTPUT_DIR, f"3d_surface_noise_{noise}.html")
        fig.write_html(out_html)
        print("Saved interactive:", out_html)

interactive_3d_surfaces(df)


Saved interactive: /home/akshy_grp12/3d_surface_noise_0.0.html
Saved interactive: /home/akshy_grp12/3d_surface_noise_0.05.html
Saved interactive: /home/akshy_grp12/3d_surface_noise_0.1.html
Saved interactive: /home/akshy_grp12/3d_surface_noise_0.15.html
Saved interactive: /home/akshy_grp12/3d_surface_noise_0.2.html
Saved interactive: /home/akshy_grp12/3d_surface_noise_0.25.html
Saved interactive: /home/akshy_grp12/3d_surface_noise_0.3.html
Saved interactive: /home/akshy_grp12/3d_surface_noise_0.35.html
Saved interactive: /home/akshy_grp12/3d_surface_noise_0.4.html
Saved interactive: /home/akshy_grp12/3d_surface_noise_0.45.html


In [8]:
summary = []
for noise in sorted(df['train_noise'].unique()):
    sub = df[df['train_noise'] == noise]
    best = sub.loc[sub['accuracy'].idxmax()]
    summary.append({
        'train_noise': noise,
        'depth': int(best['depth']),
        'mlp_ratio': best['mlp_ratio'],
        'accuracy': best['accuracy'],
        'auc_mean': best.get('auc_mean', np.nan),
        'f1': best.get('f1', np.nan)
    })
best_df = pd.DataFrame(summary).round(3)
print(best_df.to_string(index=False))
best_df.to_csv(os.path.join(OUTPUT_DIR, "best_per_noise.csv"), index=False)
print("Saved best-per-noise CSV")


 train_noise  depth  mlp_ratio  accuracy  auc_mean     f1
        0.00      8        3.0     79.43    97.748 79.379
        0.05     10        3.5     75.54    97.090 75.646
        0.10     14        1.0     69.08    95.837 69.327
        0.15     10        1.0     63.86    94.412 64.202
        0.20     12        0.5     59.29    93.199 59.619
        0.25     16        0.5     57.42    92.243 57.683
        0.30     14        1.0     54.25    91.061 54.800
        0.35     14        0.5     54.58    90.872 54.647
        0.40      8        0.5     52.11    90.200 52.230
        0.45      6        1.0     49.48    88.893 49.265
Saved best-per-noise CSV


In [9]:
import seaborn as sns
def heatmaps_per_noise(df):
    try:
        import seaborn as sns
    except:
        import sys
        !{sys.executable} -m pip install seaborn
        import seaborn as sns

    for nv in sorted(df['train_noise'].unique()):
        sub = df[df['train_noise']==nv]
        Z = sub.pivot(index='mlp_ratio', columns='depth', values='accuracy')
        plt.figure(figsize=(8,6))
        sns.heatmap(Z, annot=True, fmt=".2f", cbar_kws={'label':'Acc (%)'})
        plt.title(f"Heatmap Acc — train_noise={nv}")
        plt.xlabel('Depth'); plt.ylabel('MLP Ratio')
        out = os.path.join(OUTPUT_DIR, f"heatmap_noise_{nv}.png")
        plt.savefig(out, dpi=300, bbox_inches='tight')
        plt.close()
        print("Saved:", out)

heatmaps_per_noise(df)


Saved: /home/akshy_grp12/heatmap_noise_0.0.png
Saved: /home/akshy_grp12/heatmap_noise_0.05.png
Saved: /home/akshy_grp12/heatmap_noise_0.1.png
Saved: /home/akshy_grp12/heatmap_noise_0.15.png
Saved: /home/akshy_grp12/heatmap_noise_0.2.png
Saved: /home/akshy_grp12/heatmap_noise_0.25.png
Saved: /home/akshy_grp12/heatmap_noise_0.3.png
Saved: /home/akshy_grp12/heatmap_noise_0.35.png
Saved: /home/akshy_grp12/heatmap_noise_0.4.png
Saved: /home/akshy_grp12/heatmap_noise_0.45.png
